In [2]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier,CatBoostRegressor
from sklearn.model_selection import train_test_split

In [ ]:
train = pd.read_csv("train.csv")



In [33]:
test = pd.read_csv("test.csv")
test1 = pd.read_csv("test.csv",usecols=['ID'])

In [39]:
#train.isnull().sum(axis=0)/train.shape[0]
test = []

In [38]:
train['siteid'].fillna(-999, inplace=True)
#test['siteid'].fillna(-999, inplace=True)

train['browserid'].fillna("None", inplace=True)
#test['browserid'].fillna("None", inplace=True)

train['devid'].fillna("None", inplace=True)
#test['devid'].fillna("None", inplace=True)

C:\Users\hp pc\Anaconda3b\lib\site-packages\pandas\core\generic.py:3549: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [40]:
train['datetime'] = pd.to_datetime(train['datetime'])
#test['datetime'] = pd.to_datetime(test['datetime'])

In [41]:
# create datetime variable
train['tweekday'] = train['datetime'].dt.weekday
train['thour'] = train['datetime'].dt.hour
train['tminute'] = train['datetime'].dt.minute

#test['tweekday'] = test['datetime'].dt.weekday
#test['thour'] = test['datetime'].dt.hour
#test['tminute'] = test['datetime'].dt.minute
train.head()

,ID,datetime,siteid,offerid,category,merchant,countrycode,browserid,devid,tweekday,thour,tminute
2000000,IDr0EBu,2017-01-23 18:36:07,8600547.0,459561,48430,39772159,d,InternetExplorer,Desktop,0,18,36
2000001,IDmhaBV,2017-01-22 13:39:35,2473364.0,708906,73248,17488931,e,Edge,Tablet,6,13,39
2000002,ID7F1IP,2017-01-21 16:45:44,824014.0,375031,42227,49384126,c,Mozilla,None,5,16,45
2000003,IDexkFa,2017-01-22 16:44:57,-999.0,892449,93286,27723861,c,Mozilla Firefox,Desktop,6,16,44
2000004,ID6Qgwh,2017-01-22 15:59:49,-999.0,865605,42227,13742268,b,Mozilla Firefox,Desktop,6,15,59


In [ ]:
train['click'].mean()
a = train.groupby('devid')
a['click'].agg([np.sum, np.mean, np.std])
a = train.groupby('countrycode')
a['click'].agg([np.sum, np.mean, np.std])


In [42]:
cols = ['siteid','offerid','category','merchant']

for x in cols:
    train[x] = train[x].astype('object')
    #test[x] = test[x].astype('object')
#hhh    
#cols_to_use = list(set(train.columns) - set(['ID','datetime','click','tweekday','tminute']))

# catboost accepts categorical variables as indexes
cat_cols = [0,1,2,3,4,5,6,7]
train.head()

,ID,datetime,siteid,offerid,category,merchant,countrycode,browserid,devid,tweekday,thour,tminute
2000000,IDr0EBu,2017-01-23 18:36:07,8.60055e+06,459561,48430,39772159,d,InternetExplorer,Desktop,0,18,36
2000001,IDmhaBV,2017-01-22 13:39:35,2.47336e+06,708906,73248,17488931,e,Edge,Tablet,6,13,39
2000002,ID7F1IP,2017-01-21 16:45:44,824014,375031,42227,49384126,c,Mozilla,None,5,16,45
2000003,IDexkFa,2017-01-22 16:44:57,-999,892449,93286,27723861,c,Mozilla Firefox,Desktop,6,16,44
2000004,ID6Qgwh,2017-01-22 15:59:49,-999,865605,42227,13742268,b,Mozilla Firefox,Desktop,6,15,59


In [36]:

# modeling on sampled (1e6) rows
#rows = np.random.choice(train.index.values, 100000)
#sampled_train = train.loc[rows]
#train.index.values

In [10]:
trainX = train[cols_to_use]
trainY = train['click']
trainX.head()

,browserid,category,merchant,offerid,siteid,countrycode,thour,devid
0,Firefox,17714,20301556,887235,4.7097e+06,e,9,None
1,Mozilla Firefox,21407,9434818,178235,5.18947e+06,b,17,Desktop
2,Edge,25085,2050923,518539,98480,a,12,None
3,Firefox,40339,72089744,390352,8.8964e+06,c,10,Mobile
4,Mozilla Firefox,12052,39507200,472937,5.63512e+06,d,16,Desktop


In [11]:
X_train, X_test, y_train, y_test = train_test_split(trainX, trainY, test_size = 0.5)
model = CatBoostClassifier(depth=10, iterations=20, learning_rate=0.05, eval_metric='AUC', random_seed=1)
#X_train.head


In [12]:
model.fit(X_train
          ,y_train
          ,cat_features=cat_cols
          ,eval_set = (X_test, y_test)
          ,use_best_model = True
         )

In [44]:
pred = model.predict_proba(test[cols_to_use])[:,1]

In [3]:

#test[cols_to_use]

In [200]:
pred

array([ 0.18962573,  0.13338629,  0.02924319, ...,  0.02311152,
        0.00806764,  0.11363434])

0.012510717304491764

In [46]:
sub = pd.DataFrame({'ID':test['ID'],'click':pred})
sub.to_csv('b2.csv',index=False)

In [211]:
#test = pd.read_csv("test.csv")

In [49]:
import pandas as pd

In [50]:
import glob,os

In [51]:
os.chdir(r"C:\Users\hp pc\Desktop\computer visi\multilab")

In [59]:
results = pd.DataFrame([])

In [62]:
for counter, file in enumerate(glob.glob("b*")):
    namedf = pd.read_csv(file, skiprows=0)
    results = results.append(namedf)
 
results.to_csv(r'C:\Users\hp pc\Desktop\computer visi\multilab\ab34.csv')

In [64]:
a = pd.read_csv("ab34.csv")

In [65]:
test=[]

In [67]:
a = a[['ID','click']]

In [77]:

results.to_csv(r'C:\Users\hp pc\Desktop\computer visi\multilab\ab34521.csv',index=False)

In [78]:
a = pd.read_csv("ab34521.csv")

In [80]:
a=[]